In [3]:
import numpy as np
import random 

In [ ]:
class Sudoku:
	def __init__(self, ):
		self.matrix = np.zeros((9,9), dtype=np.int8)
		self.list_numbers = list(range(1,10))
		self.end = False
		self.dificult = 'easy'
	
	def level(self,):
		if self.dificult == 'easy':
			self.luck_percent = .45
		elif self.dificult == 'medium':
			self.luck_percent = .325
		else: self.luck_percent = .25


	def fill_matrix(self):
		for i, row in enumerate(self.matrix):
			for j, cel in enumerate(row):
				if random.random() > self.luck_percent:
					self.matrix[i][j] = int(cel)
				else:
					self.matrix[i][j] = int(random.choice(self.list_numbers))

	def show_matrix(self,):
		cont_row, cont_col = 0, 0
		print("=== Show Matrix ===")
		print('_______________________',)
		for row in self.matrix:
			print('| ', end='')
			for cel in row:
				print(cel, end=' ')
				cont_row += 1
				if cont_row == 3:
					cont_row = 0
					print('| ', end='')

			cont_col += 1
			if cont_col == 3:
				cont_col = 0
				print('')
				print('_______________________',)

			print('')
	
	def run(self):
		print('=== Welcome to the game ===')
		# self.dificult = input('Choice your difficult: "easy", "medium" or "hard": ')
		self.dificult = 'easy'
		self.level()
		self.fill_matrix()
		self.show_matrix()

		# while self.end == False:
			
		# 	self.show_matrix()

		# 	break



			

partida = Sudoku()

partida.run()
# for i in partida.matrix:
# 	print(i)

=== Welcome to the game ===
[32 23 13 32 21 11 16 24 26]
=== Show Matrix ===
_______________________
| 9 0 0 | 0 0 0 | 6 0 0 | 
| 0 6 0 | 4 9 0 | 0 0 0 | 
| 8 5 0 | 0 0 0 | 3 7 5 | 
_______________________

| 5 4 0 | 6 0 0 | 0 4 5 | 
| 0 5 0 | 6 0 5 | 0 7 3 | 
| 2 3 8 | 8 7 5 | 0 0 4 | 
_______________________

| 5 0 0 | 0 0 1 | 0 0 1 | 
| 0 0 5 | 0 0 0 | 5 6 8 | 
| 3 0 0 | 8 5 0 | 2 0 0 | 
_______________________



In [5]:
list_numbers = list(range(1,10))


In [6]:
print(random.choice(list_numbers))

4


In [7]:
random.random()

0.7772607365977093